In [1]:
import os

os.environ['TORCH_HOME'] = '/disks/sdb/torch_home'

from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithLMHead.from_pretrained("bert-base-uncased")


I0226 09:35:21.231179 140631251642176 file_utils.py:38] PyTorch version 1.4.0 available.
I0226 09:35:24.576916 140631251642176 filelock.py:274] Lock 140631136788832 acquired on /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock
I0226 09:35:24.580226 140631251642176 file_utils.py:413] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /disks/sdb/torch_home/transformers/tmpd0z910re


I0226 09:35:25.907350 140631251642176 file_utils.py:423] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0226 09:35:25.908533 140631251642176 file_utils.py:426] creating metadata file for /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0226 09:35:25.910006 140631251642176 filelock.py:318] Lock 140631136788832 released on /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock
I0226 09:35:25.911094 140631251642176 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config

I0226 09:35:27.153762 140631251642176 tokenization_utils.py:418] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /disks/sdb/torch_home/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0226 09:35:28.613382 140631251642176 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0226 09:35:28.615016 140631251642176 configuration_utils.py:290] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_drop

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help lower our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help minimize our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.


In [ ]:
class BertAugmentor:
    def __init__(self):
        pass
    
    def augment():
        pass

In [131]:
import random
from collections import defaultdict
from copy import deepcopy
import numpy as np

sent = "the large versions would help minimize our carbon footprint."
aug_num = 2
change_num = 3
top_k = 5
temprature = 1.0
stopwords = ['a', 'the', 'our', ',', '.', '?']

sent_split = sent.split(" ")

allow_sids = []
for i in range(len(sent_split)):
    if sent_split[i] not in stopwords:
        allow_sids.append(i)

change_num = min(change_num, len(allow_sids))
change_sids_lst = []
inputs = []
sent_splits = []
for i in range(aug_num):
    sids = random.sample(allow_sids, k=change_num)
#     sids = [0, 2, 4]
    change_sids_lst.append(sids)
    tmp_sent_split = deepcopy(sent_split)
    sent_splits.append(tmp_sent_split)
    for sid in sids:
        tmp_sent_split[sid] = tokenizer.mask_token
    inputs.append(tokenizer.encode(tmp_sent_split, return_tensors="pt"))
inputs = torch.cat(inputs)

mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]
with torch.no_grad():
    token_logits = model(inputs)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_k_logits, top_k_tokens = torch.topk(mask_token_logits, top_k, dim=1)

top_k_logits = top_k_logits.view(aug_num, change_num, top_k) / temprature
top_k_probs = torch.softmax(top_k_logits, dim=-1).cpu().numpy()
top_k_tokens = top_k_tokens.view(aug_num, change_num, top_k).cpu().numpy().tolist()

print(top_k_tokens)

for i in range(aug_num):
    for j in range(change_num):
        changed = np.random.choice(top_k_tokens[i][j], 1, p=top_k_probs[i][j])
        print(changed)
#         print(sent_splits[i][j])
        sent_splits[i][change_sids_lst[i][j]] = tokenizer.decode([changed])

sents = list(map(lambda x: " ".join(x), sent_splits))
sents

[[[2047, 2101, 2060, 2048, 10947], [6409, 3891, 3465, 5366, 4254], [1012, 1025, 1029, 999, 1010]], [[2393, 2022, 2079, 5335, 2031], [6409, 3891, 3465, 5366, 4254], [1012, 1025, 1029, 999, 1010]]]
[2047]
[6409]
[1012]
[2393]
[5366]
[1012]


['the new versions would help minimize our . losses',
 'the large versions would costs minimize our . help']

In [3]:
sequence = " of the large versions would help minimize our carbon footprint."
inputs = tokenizer.encode(sequence, return_tensors="pt")


mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

token_logits = model(inputs)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

 of the large versions would help reduce our carbon footprint.
 of the large versions would help lower our carbon footprint.
 of the large versions would help minimize our carbon footprint.
 of the large versions would help increase our carbon footprint.
 of the large versions would help decrease our carbon footprint.


In [103]:
np.random.choice(top_k_tokens[0][0], 1, p=top_k_probs[0][0])
# top_k_probs[0][0]
# help(np.random.choice)

array([2008])

In [2]:
import os

os.environ['TORCH_HOME'] = '/disks/sdb/torch_home'

import torch
bart = torch.hub.load('pytorch/fairseq', 'bart.large').cuda()
bart.eval()  # disable dropout (or leave in train mode to finetune)

Using cache found in /disks/sdb/torch_home/hub/pytorch_fairseq_master


BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [3]:
hypotheses_batch = bart.sample(['slines'], beam=4, lenpen=2.0, max_len_b=140, min_len=55, no_repeat_ngram_size=3)

AttributeError: 'DenoisingTask' object has no attribute 'build_dataset_for_inference'